<a href="https://colab.research.google.com/github/nczarli/f-jl221-2/blob/main/soft_teacher_custom_dataset_300_iter_less_data_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nczarli/f-jl221-2.git
!git clone https://github.com/nczarli/SoftTeacherCustomDataset.git

Cloning into 'f-jl221-2'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 149 (delta 70), reused 86 (delta 36), pack-reused 0
Receiving objects: 100% (149/149), 7.78 MiB | 6.37 MiB/s, done.
Resolving deltas: 100% (70/70), done.
Cloning into 'SoftTeacherCustomDataset'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 343 (delta 137), reused 133 (delta 115), pack-reused 148
Receiving objects: 100% (343/343), 596.32 KiB | 18.07 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [2]:
ls

f-jl221-2/  sample_data/  SoftTeacherCustomDataset/


In [3]:
cd SoftTeacherCustomDataset/

/content/SoftTeacherCustomDataset


In [4]:
ls

configs/  LICENSE   README.md         resources/   setup.py  tools/
demo/     Makefile  requirements.txt  SECURITY.md  ssod/


In [5]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.5 MB/s eta 0:13:50tcmalloc: large alloc 1147494400 bytes == 0x393c4000 @  0x7f76c6fd4615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:15tcmalloc: large alloc 1434370048 bytes == 0x7da1a000 @  0x7f76c6fd4615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [6]:
pip install mmcv-full==1.3.17 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 50.4 MB 10.2 MB/s 
     |████████████████████████████████| 190 kB 24.0 MB/s 


In [7]:
!make install

make pre
make[1]: Entering directory '/content/SoftTeacherCustomDataset'
python -m pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1.9 MB 31.0 MB/s 
     |████████████████████████████████| 182 kB 76.2 MB/s 
     |████████████████████████████████| 166 kB 74.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 78.4 MB/s 
     |████████████████████████████████| 162 kB 76.5 MB/s 
     |████████████████████████████████| 162 kB 79.8 MB/s 
     |████████████████████████████████| 158 kB 81.0 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |█████████████████████

In [8]:
cd ..

/content


In [9]:
ls

f-jl221-2/  sample_data/  SoftTeacherCustomDataset/


In [10]:
!cp -R f-jl221-2/labels_generated/ SoftTeacherCustomDataset/labels_generated

In [11]:
cd SoftTeacherCustomDataset/

/content/SoftTeacherCustomDataset


In [12]:
ls

configs/           LICENSE    requirements.txt  setup.py        thirdparty/
demo/              Makefile   resources/        ssod/           tools/
labels_generated/  README.md  SECURITY.md       ssod.egg-info/


In [13]:
# Preprocess the images, scale y axis by 1.5

# find all bmp images in labels_generated folder
# for each image resize y axis by 1.5 times and overwrite the image

import os
import cv2
import numpy as np

# get all bmp files in labels_generated and subfolders
bmp_files = []

for root, dirs, files in os.walk("labels_generated"):
    for file in files:
        if file.endswith(".bmp"):
             bmp_files.append(os.path.join(root, file))
print(len(bmp_files))
# resize each bmp file
for bmp_file in bmp_files:
    img = cv2.imread(bmp_file, cv2.IMREAD_GRAYSCALE)
    print(img.shape)
    img = cv2.resize(img, (0,0), fx=1, fy=1.5)
    print(img.shape)
    cv2.imwrite(bmp_file, img)

31
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)
(500, 448)
(750, 448)


In [14]:
!bash tools/dist_train.sh configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels.py 1

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : tools/train.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_0mjg05na/none_fpv95sad
INFO:torch.distributed.elastic.agent.server.api:[default] starti

# Inference

## Add paths of thirdparty packages

In [15]:
import sys
sys.path.insert(0,'/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet')

In [16]:
import sys
print(sys.path)

['/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [17]:
import sys
sys.path.insert(0,' /content/SoftTeacherCustomDataset')

In [18]:
!export PYTHONPATH=$(pwd)

In [19]:
cd /content/SoftTeacherCustomDataset/thirdparty/mmdetection/

/content/SoftTeacherCustomDataset/thirdparty/mmdetection


In [20]:
import mmdet

In [21]:
cd /content

/content


## Visualise

In [22]:
"""%matplotlib inline 
import mmcv
from matplotlib import pyplot as plt
from mmdet.apis import init_detector, inference_detector

config_file = 'SoftTeacherCustomDataset/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels.py'
checkpoint_file = 'SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels/latest.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img = '/content/SoftTeacherCustomDataset/labels_generated/test/images/Cherries,24,19-08-2022,18-10-52,pass.bmp'  # or img = mmcv.imread(img), which will only load it once
result = inference_detector(model, img)
# visualize the results in a new window
data = model.show_result(img, result)"""


"%matplotlib inline \nimport mmcv\nfrom matplotlib import pyplot as plt\nfrom mmdet.apis import init_detector, inference_detector\n\nconfig_file = 'SoftTeacherCustomDataset/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels.py'\ncheckpoint_file = 'SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels/latest.pth'\n\n# build the model from a config file and a checkpoint file\nmodel = init_detector(config_file, checkpoint_file, device='cuda:0')\n\n# test a single image and show the results\nimg = '/content/SoftTeacherCustomDataset/labels_generated/test/images/Cherries,24,19-08-2022,18-10-52,pass.bmp'  # or img = mmcv.imread(img), which will only load it once\nresult = inference_detector(model, img)\n# visualize the results in a new window\ndata = model.show_result(img, result)"

In [23]:
"""plt.figure(figsize = (20,40))
plt.imshow(data, interpolation='nearest')"""

"plt.figure(figsize = (20,40))\nplt.imshow(data, interpolation='nearest')"

In [24]:
!python SoftTeacherCustomDataset/demo/image_demo.py /content/SoftTeacherCustomDataset/labels_generated/test/images/Cherries,24,19-08-2022,18-10-52,pass.bmp SoftTeacherCustomDataset/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels.py SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels/latest.pth --output work_dirs/

load checkpoint from local path: SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels/latest.pth
/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Save results to work_dirs/Cherries,24,19-08-2022,18-10-52,pass.bmp


In [25]:
cd SoftTeacherCustomDataset/

/content/SoftTeacherCustomDataset


In [26]:
!bash tools/dist_test.sh configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels.py work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels/latest.pth 1 --eval bbox --cfg-options model.test_cfg.rcnn.score_thr=0.0001


/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : tools/test.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_afcr7waj/none_p2l518l1
INFO:torch.distributed.elastic.agent.server.api:[default] startin